In [97]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import math

In [98]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Neural Network 

In [99]:
class DeepONet(nn.Module):
    def __init__(self):
        super(DeepONet,self).__init__()
        self.Bralch_L1 = nn.Linear(100,20)
        self.Bralch_L2 = nn.Linear(20,20)
        self.Bralch_L3 = nn.Linear(20,20)
        self.Bralch_L4 = nn.Linear(20,20)
        self.Bralch_L5 = nn.Linear(20,100)

        self.Trunk_L1 = nn.Linear(2,20)
        self.Trunk_L2 = nn.Linear(20,20)
        self.Trunk_L3 = nn.Linear(20,20)
        self.Trunk_L4 = nn.Linear(20,20)
        self.Trunk_L5 = nn.Linear(20,100)
        
    def forward(self,y_0,x_loc_and_time):
        # Branch
        b = F.relu(self.Bralch_L1(y_0))
        b = F.relu(self.Bralch_L2(b))
        b = F.relu(self.Bralch_L3(b))
        b = F.relu(self.Bralch_L4(b))
        b = self.Bralch_L5(b)

        # Trunk 
        t = F.relu(self.Trunk_L1(x_loc_and_time))
        t = F.relu(self.Trunk_L2(t))
        t = F.relu(self.Trunk_L3(t))
        t = F.relu(self.Trunk_L4(t))
        t = self.Trunk_L5(t)

        # dot product
        output = torch.sum(b*torch.(t),dim=0) 

        return output

In [100]:
df_trial = pd.read_csv("Dataset_1.csv").iloc[:,1:-3].dropna().to_numpy(dtype='float32')

In [101]:
df_trial.shape

(141000, 103)

In [102]:
class Data(Dataset):
    def __init__(self,transform=None):
        data_full = df_trial
        self.init_conditions = torch.from_numpy(data_full[:,:100])
        self.x_loc = torch.from_numpy(data_full[:,[100]])
        self.Time = torch.from_numpy(data_full[:,[101]])
        self.Y_value = torch.from_numpy(data_full[:,[102]])
        self.n_samples = data_full.shape[0]

    def __getitem__(self, index):
        return self.init_conditions[index] , self.x_loc[index] , self.Time[index] , self.Y_value[index] 
    
    def __len__(self):
        return self.n_samples

In [103]:
model = DeepONet().to(device)

In [104]:
device

device(type='cuda')

In [105]:
train_size = int(0.7*len(Data()))
test_size = len(Data()) - train_size

In [106]:
Burger_train , Burger_split = torch.utils.data.random_split(Data(), [train_size,test_size])

In [107]:
data_train = Burger_train
train_loader = DataLoader(dataset=data_train,batch_size=300,shuffle=True)

In [108]:
data_iter = iter(train_loader)
data = data_iter.__next__()
Init_val , x_loc, time, y_value = data
print(y_value.shape)

torch.Size([300, 1])


In [109]:
num_epoch = 20
total_samples = len(train_loader)
n_iterations = math.ceil(total_samples/300)
print(total_samples,n_iterations)
learning_rate = 0.01

329 2


In [110]:
criterion = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [111]:
for epoch in range(num_epoch):
    for i , (input_init_conditions,input_x_loc,input_time,Actual_y) in enumerate(train_loader):
        input1 = input_init_conditions
        input1 = input1.to(device)

        input2 = torch.cat((input_x_loc,input_time),-1)
        input2 = input2.to(device)

        Actual_y = Actual_y.to(device)

        Outputs = model(input1,input2)
        loss = criterion(Outputs,Actual_y)


        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if (i+1) % 100 ==0:
            print(f'Epoch [{epoch+1}/{num_epoch}] , Step [{i+1}/{total_samples}] , Loss: {loss.item():.16f}')

TypeError: transpose() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim0, int dim1)
 * (Tensor input, name dim0, name dim1)


In [112]:
model(input1,input2)

TypeError: transpose() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim0, int dim1)
 * (Tensor input, name dim0, name dim1)


In [96]:
print(input1.device)        # Should output cuda:0 or cpu
print(input2.device)        # Should output cuda:0 or cpu
print(next(model.parameters()).device) 

cuda:0
cuda:0
cuda:0
